# Amazon SageMaker Autopilot Candidate Definition Notebook

This notebook was automatically generated by the AutoML job **Autopilot-HAC-DEMO**.
This notebook allows you to customize the candidate definitions and execute the SageMaker Autopilot workflow.

The dataset has **562** columns and the column named **activity** is used as
the target column. This is being treated as a **MulticlassClassification** problem. The dataset also has **6** classes.
This notebook will build a **[MulticlassClassification](https://en.wikipedia.org/wiki/Multiclass_classification)** model that
**maximizes** the "**ACCURACY**" quality metric of the trained models.
The "**ACCURACY**" metric provides the percentage of times the model predicted the correct class.

As part of the AutoML job, the input dataset has been randomly split into two pieces, one for **training** and one for
**validation**. This notebook helps you inspect and modify the data transformation approaches proposed by Amazon SageMaker Autopilot. You can interactively
train the data transformation models and use them to transform the data. Finally, you can execute a multiple algorithm hyperparameter optimization (multi-algo HPO)
job that helps you find the best model for your dataset by jointly optimizing the data transformations and machine learning algorithms.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>
Look for sections like this for recommended settings that you can change.
</div>


---

## Contents

1. [Sagemaker Setup](#Sagemaker-Setup)
    1. [Downloading Generated Candidates](#Downloading-Generated-Modules)
    1. [SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration](#SageMaker-Autopilot-Job-and-Amazon-Simple-Storage-Service-(Amazon-S3)-Configuration)
1. [Candidate Pipelines](#Candidate-Pipelines)
    1. [Generated Candidates](#Generated-Candidates)
    1. [Selected Candidates](#Selected-Candidates)
1. [Executing the Candidate Pipelines](#Executing-the-Candidate-Pipelines)
    1. [Run Data Transformation Steps](#Run-Data-Transformation-Steps)
    1. [Multi Algorithm Hyperparameter Tuning](#Multi-Algorithm-Hyperparameter-Tuning)
1. [Model Selection and Deployment](#Model-Selection-and-Deployment)
    1. [Tuning Job Result Overview](#Tuning-Job-Result-Overview)
    1. [Model Deployment](#Model-Deployment)

---

## Sagemaker Setup

Before you launch the SageMaker Autopilot jobs, we'll setup the environment for Amazon SageMaker
- Check environment & dependencies.
- Create a few helper objects/function to organize input/output data and SageMaker sessions.

**Minimal Environment Requirements**

- Jupyter: Tested on `JupyterLab 1.0.6`, `jupyter_core 4.5.0` and `IPython 6.4.0`
- Kernel: `conda_python3`
- Dependencies required
  - `sagemaker-python-sdk>=v1.43.4`
- Expected Execution Role/permission
  - S3 access to the bucket that stores the notebook.

### Downloading Generated Modules
Download the generated data transformation modules and an SageMaker Autopilot helper module used by this notebook.
Those artifacts will be downloaded to **Autopilot-HAC-DEMO-artifacts** folder.

In [1]:
!mkdir -p Autopilot-HAC-DEMO-artifacts
!aws s3 sync s3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO/sagemaker-automl-candidates/pr-1-3c1dec4b63f54b22875bc400ae56d8cef2d39147a7724381bf67ee26be/generated_module Autopilot-HAC-DEMO-artifacts/generated_module --only-show-errors
!aws s3 sync s3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO/sagemaker-automl-candidates/pr-1-3c1dec4b63f54b22875bc400ae56d8cef2d39147a7724381bf67ee26be/notebooks/sagemaker_automl Autopilot-HAC-DEMO-artifacts/sagemaker_automl --only-show-errors

import sys
sys.path.append("Autopilot-HAC-DEMO-artifacts")

### SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration

The following configuration has been derived from the SageMaker Autopilot job. These items configure where this notebook will
look for generated candidates, and where input and output data is stored on Amazon S3.

In [2]:
from sagemaker_automl import uid, AutoMLLocalRunConfig

# Where the preprocessed data from the existing AutoML job is stored
BASE_AUTOML_JOB_NAME = 'Autopilot-HAC-DEMO'
BASE_AUTOML_JOB_CONFIG = {
    'automl_job_name': BASE_AUTOML_JOB_NAME,
    'automl_output_s3_base_path': 's3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO',
    'data_transformer_image_repo_version': '0.1.0-cpu-py3',
    'algo_image_repo_versions': {'xgboost': '0.90-1-cpu-py3', 'linear-learner': 'latest'}
}

# Path conventions of the output data storage path from the local AutoML job run of this notebook
LOCAL_AUTOML_JOB_NAME = 'Autopilot--notebook-run-{}'.format(uid())
LOCAL_AUTOML_JOB_CONFIG = {
    'local_automl_job_name': LOCAL_AUTOML_JOB_NAME,
    'local_automl_job_output_s3_base_path': 's3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO/{}'.format(LOCAL_AUTOML_JOB_NAME),
    'data_processing_model_dir': 'data-processor-models',
    'data_processing_transformed_output_dir': 'transformed-data',
    'multi_algo_tuning_output_dir': 'multi-algo-tuning'
}

AUTOML_LOCAL_RUN_CONFIG = AutoMLLocalRunConfig(
    role='arn:aws:iam::563487891580:role/service-role/AmazonSageMaker-ExecutionRole-20200208T102977',
    base_automl_job_config=BASE_AUTOML_JOB_CONFIG,
    local_automl_job_config=LOCAL_AUTOML_JOB_CONFIG,
    security_config={'EnableInterContainerTrafficEncryption': False, 'VpcConfig': {}})

AUTOML_LOCAL_RUN_CONFIG.display()

This notebook is initialized to use the following configuration: 
        <table>
        <tr><th colspan=2>Name</th><th>Value</th></tr>
        <tr><th>General</th><th>Role</th><td>arn:aws:iam::563487891580:role/service-role/AmazonSageMaker-ExecutionRole-20200208T102977</td></tr>
        <tr><th rowspan=2>Base AutoML Job</th><th>Job Name</th><td>Autopilot-HAC-DEMO</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO</td></tr>
        <tr><th rowspan=5>Interactive Job</th><th>Job Name</th><td>Autopilot--notebook-run-01-15-10-53</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO/Autopilot--notebook-run-01-15-10-53</td></tr>
        <tr><th>Data Processing Trained Model Directory</th><td>s3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO/Autopilot--notebook-run-01-15-10-53/data-processor-models</td></tr>
        <tr><th>Data Processing Transformed Output</th><td>s3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO/Autopilot--notebook-run-01-15-10-53/transformed-data</td></tr>
        <tr><th>Algo Tuning Model Output Directory</th><td>s3://sagemaker-us-east-2-563487891580/sagemaker/DEMO-autopilot-had-dataset/output/Autopilot-HAC-DEMO/Autopilot--notebook-run-01-15-10-53/multi-algo-tuning</td></tr>
        </table>
        

## Candidate Pipelines

The `AutoMLLocalRunner` keeps track of selected candidates and automates many of the steps needed to execute feature engineering and tuning steps.

In [3]:
from sagemaker_automl import AutoMLInteractiveRunner, AutoMLLocalCandidate

automl_interactive_runner = AutoMLInteractiveRunner(AUTOML_LOCAL_RUN_CONFIG)

### Generated Candidates

The SageMaker Autopilot Job has analyzed the dataset and has generated **10** machine learning
pipeline(s) that use **2** algorithm(s). Each pipeline contains a set of feature transformers and an
algorithm.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. The resource configuration: instance type & count
1. Select candidate pipeline definitions by cells
1. The linked data transformation script can be reviewed and updated. Please refer to the [README.md](./Autopilot-HAC-DEMO-artifacts/generated_module/README.md) for detailed customization instructions.
</div>

**[dpp0-xgboost](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp0.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [4]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp0",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp1-linear-learner](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp1.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [5]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp1",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp2-linear-learner](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp2.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [6]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp2",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp3-xgboost](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp3.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [7]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp3",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp4-linear-learner](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp4.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [8]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp4",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp5-linear-learner](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp5.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [9]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp5",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp6-linear-learner](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp6.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [10]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp6",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp7-xgboost](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp7.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [11]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp7",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp8-xgboost](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp8.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [12]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp8",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

**[dpp9-xgboost](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp9.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [13]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp9",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        }
    }
})

### Selected Candidates

You have selected the following candidates (please run the cell below and click on the feature transformer links for details):

In [14]:
automl_interactive_runner.display_candidates()

Candidate Name,Algorithm,Feature Transformer
dpp0-xgboost,xgboost,dpp0.py
dpp1-linear-learner,linear-learner,dpp1.py
dpp2-linear-learner,linear-learner,dpp2.py
dpp3-xgboost,xgboost,dpp3.py
dpp4-linear-learner,linear-learner,dpp4.py
dpp5-linear-learner,linear-learner,dpp5.py
dpp6-linear-learner,linear-learner,dpp6.py
dpp7-xgboost,xgboost,dpp7.py
dpp8-xgboost,xgboost,dpp8.py
dpp9-xgboost,xgboost,dpp9.py


The feature engineering pipeline consists of two SageMaker jobs:

1. Generated trainable data transformer Python modules like [dpp0.py](Autopilot-HAC-DEMO-artifacts/generated_module/candidate_data_processors/dpp0.py), which has been downloaded to local file system
2. A **training** job to train the data transformers
3. A **batch transform** job to apply the trained transformation to the dataset to generate the algorithm compatible data

The transformers and its training pipeline are built using open sourced **[sagemaker-scikit-learn-container][]** and **[sagemaker-scikit-learn-extension][]**.

[sagemaker-scikit-learn-container]: https://github.com/aws/sagemaker-scikit-learn-container
[sagemaker-scikit-learn-extension]: https://github.com/aws/sagemaker-scikit-learn-extension

## Executing the Candidate Pipelines

Each candidate pipeline consists of two steps, feature transformation and algorithm training.
For efficiency first execute the feature transformation step which will generate a featurized dataset on S3
for each pipeline.

After each featurized dataset is prepared, execute a multi-algorithm tuning job that will run tuning jobs
in parallel for each pipeline. This tuning job will execute training jobs to find the best set of
hyper-parameters for each pipeline, as well as finding the overall best performing pipeline.

### Run Data Transformation Steps

Now you are ready to start execution all data transformation steps.  The cell below may take some time to finish,
feel free to go grab a cup of coffee. To expedite the process you can set the number of `parallel_jobs` to be up to 10.
Please check the account limits to increase the limits before increasing the number of jobs to run in parallel.

In [15]:
automl_interactive_runner.fit_data_transformers(parallel_jobs=7)

2020-03-01 15:11:00,039 INFO root: [Worker_1:dpp1-linear-learner]Executing step: train_data_transformer
2020-03-01 15:11:00,513 INFO sagemaker: Creating training-job with name: Autopilot--notebook-run-01-15-10-53-dpp1-train-01-15-10-57

2020-03-01 15:11:00 Starting - Starting the training job2020-03-01 15:11:01,039 INFO root: [Worker_2:dpp2-linear-learner]Executing step: train_data_transformer
2020-03-01 15:11:01,040 INFO root: [Worker_3:dpp3-xgboost]Executing step: train_data_transformer
2020-03-01 15:11:01,040 INFO root: [Worker_0:dpp0-xgboost]Executing step: train_data_transformer
2020-03-01 15:11:01,466 INFO sagemaker: Creating training-job with name: Autopilot--notebook-run-01-15-10-53-dpp2-train-01-15-10-57
2020-03-01 15:11:01,475 INFO sagemaker: Creating training-job with name: Autopilot--notebook-run-01-15-10-53-dpp3-train-01-15-10-57
2020-03-01 15:11:01,484 INFO sagemaker: Creating training-job with name: Autopilot--notebook-run-01-15-10-53-dpp0-train-01-15-10-57

2020-03-01 1

### Multi Algorithm Hyperparameter Tuning

Now that the algorithm compatible trasformed datasets are ready, you can start the multi-algorithm model tuning job
to find the best predictive model. The following algorithm training job configuration for each
algorithm is auto-generated by the AutoML Job as part of the recommendation.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Hyperparameter ranges
2. Objective metrics
3. Recommended static algorithm hyperparameters.

Please refers to [Xgboost tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) and [Linear learner tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner-tuning.html) for detailed explanations of the parameters.
</div>

The AutoML recommendation job has recommended the following hyperparameters, objectives and accuracy metrics for
the algorithm and problem type:

In [16]:
ALGORITHM_OBJECTIVE_METRICS = {
    'xgboost': 'validation:accuracy',
    'linear-learner': 'validation:multiclass_accuracy',
}

STATIC_HYPERPARAMETERS = {
    'xgboost': {
        'objective': 'multi:softmax',
        'num_class': 6,
    },
    'linear-learner': {
        'predictor_type': 'multiclass_classifier',
        'loss': 'auto',
        'mini_batch_size': 800,
        'num_classes': 6,
    },
}

The following tunable hyperparameters search ranges are recommended for the Multi-Algo tuning job:

In [17]:
from sagemaker.parameter import CategoricalParameter, ContinuousParameter, IntegerParameter

ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES = {
    'xgboost': {
        'num_round': IntegerParameter(2, 512, scaling_type='Logarithmic'),
        'max_depth': IntegerParameter(2, 32, scaling_type='Logarithmic'),
        'eta': ContinuousParameter(1e-3, 1.0, scaling_type='Logarithmic'),
        'gamma': ContinuousParameter(1e-6, 64.0, scaling_type='Logarithmic'),
        'min_child_weight': ContinuousParameter(1e-6, 32.0, scaling_type='Logarithmic'),
        'subsample': ContinuousParameter(0.5, 1.0, scaling_type='Linear'),
        'colsample_bytree': ContinuousParameter(0.3, 1.0, scaling_type='Linear'),
        'lambda': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
        'alpha': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
    },
    'linear-learner': {
        'wd': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'l1': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'learning_rate': ContinuousParameter(1e-5, 1.0, scaling_type='Logarithmic'),
    },
}

#### Prepare Multi-Algorithm Tuner Input

To use the multi-algorithm HPO tuner, prepare some inputs and parameters. Prepare a dictionary whose key is the name of the trained pipeline candidates and the values are respectively:

1. Estimators for the recommended algorithm
2. Hyperparameters search ranges
3. Objective metrics

In [18]:
multi_algo_tuning_parameters = automl_interactive_runner.prepare_multi_algo_parameters(
    objective_metrics=ALGORITHM_OBJECTIVE_METRICS,
    static_hyperparameters=STATIC_HYPERPARAMETERS,
    hyperparameters_search_ranges=ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES)

Below you prepare the inputs data to the multi-algo tuner:

In [19]:
multi_algo_tuning_inputs = automl_interactive_runner.prepare_multi_algo_inputs()

#### Create Multi-Algorithm Tuner

With the recommended Hyperparameter ranges and the transformed dataset, create a multi-algorithm model tuning job
that coordinates hyper parameter optimizations across the different possible algorithms and feature processing strategies.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Tuner strategy: [Bayesian](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization), [Random Search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)
2. Objective type: `Minimize`, `Maximize`, see [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization)
3. Max Job size: the max number of training jobs HPO would be launching to run experiments. Note the default value is **250**
    which is the default of the managed flow.
4. Parallelism. Number of jobs that will be executed in parallel. Higher value will expedite the tuning process.
    Please check the account limits to increase the limits before increasing the number of jobs to run in parallel
5. Please use a different tuning job name if you re-run this cell after applied customizations.
</div>

In [20]:
from sagemaker.tuner import HyperparameterTuner

base_tuning_job_name = "{}-tuning".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name)

tuner = HyperparameterTuner.create(
    base_tuning_job_name=base_tuning_job_name,
    strategy='Bayesian',
    objective_type='Maximize',
    max_parallel_jobs=7,
    max_jobs=250,
    **multi_algo_tuning_parameters,
)

#### Run Multi-Algorithm Tuning

Now you are ready to start running the **Multi-Algo Tuning** job. After the job is finished, store the tuning job name which you use to select models in the next section.
The tuning process will take some time, please track the progress in the Amazon SageMaker Hyperparameter tuning jobs console.

In [21]:
from IPython.display import display, Markdown

# Run tuning
tuner.fit(inputs=multi_algo_tuning_inputs, include_cls_metadata=None)
tuning_job_name = tuner.latest_tuning_job.name

display(
    Markdown(f"Tuning Job {tuning_job_name} started, please track the progress from [here](https://{AUTOML_LOCAL_RUN_CONFIG.region}.console.aws.amazon.com/sagemaker/home?region={AUTOML_LOCAL_RUN_CONFIG.region}#/hyper-tuning-jobs/{tuning_job_name})"))

# Wait for tuning job to finish
tuner.wait()

2020-03-01 15:24:09,308 INFO root: _TuningJob.start_new!!!
2020-03-01 15:24:09,312 INFO sagemaker: Creating hyperparameter tuning job with name: Autopilot--notebook--200301-1524


Tuning Job Autopilot--notebook--200301-1524 started, please track the progress from [here](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/hyper-tuning-jobs/Autopilot--notebook--200301-1524)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Model Selection and Deployment

This section guides you through the model selection process. Afterward, you construct an inference pipeline
on Amazon SageMaker to host the best candidate.

Because you executed the feature transformation and algorithm training in two separate steps, you now need to manually
link each trained model with the feature transformer that it is associated with. When running a regular Amazon
SageMaker Autopilot job, this will automatically be done for you.

### Tuning Job Result Overview

The performance of each candidate pipeline can be viewed as a Pandas dataframe. For more interactive usage please
refers to [model tuning monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html).

In [22]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

SAGEMAKER_SESSION = AUTOML_LOCAL_RUN_CONFIG.sagemaker_session
SAGEMAKER_ROLE = AUTOML_LOCAL_RUN_CONFIG.role

tuner_analytics = HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

,alpha,colsample_bytree,eta,gamma,lambda,max_depth,min_child_weight,num_round,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds,TrainingJobDefinitionName,l1,learning_rate,wd
125,0.000002,0.328032,0.139698,0.000099,0.234341,4.0,0.000103,511.0,0.519963,Autopilot--notebook--200301-1524-125-8c267365,Completed,0.988321,2020-03-01 16:20:01+00:00,2020-03-01 16:21:14+00:00,73.0,dpp7-xgboost,NaN,NaN,NaN
142,0.007444,0.300000,0.169870,0.000022,0.008090,8.0,0.019923,358.0,0.670710,Autopilot--notebook--200301-1524-108-841fcf27,Completed,0.986861,2020-03-01 16:11:13+00:00,2020-03-01 16:12:13+00:00,60.0,dpp7-xgboost,NaN,NaN,NaN
62,0.001316,0.948081,0.319048,0.005119,0.001748,2.0,0.000001,116.0,0.988348,Autopilot--notebook--200301-1524-188-3ede0ea9,Completed,0.986861,2020-03-01 16:48:35+00:00,2020-03-01 16:49:26+00:00,51.0,dpp7-xgboost,NaN,NaN,NaN
141,0.054502,0.392894,0.252448,0.000007,0.467405,4.0,0.013757,343.0,0.782205,Autopilot--notebook--200301-1524-109-4073076d,Completed,0.986861,2020-03-01 16:11:51+00:00,2020-03-01 16:12:48+00:00,57.0,dpp7-xgboost,NaN,NaN,NaN
104,0.000066,0.519955,0.673174,0.020842,0.000019,2.0,0.064072,341.0,0.851468,Autopilot--notebook--200301-1524-146-dadc4cba,Completed,0.986861,2020-03-01 16:29:41+00:00,2020-03-01 16:30:37+00:00,56.0,dpp7-xgboost,NaN,NaN,NaN
94,0.000010,0.425686,0.295590,0.000008,0.000001,2.0,0.000017,306.0,0.925790,Autopilot--notebook--200301-1524-156-e96e1a71,Completed,0.985401,2020-03-01 16:34:22+00:00,2020-03-01 16:35:13+00:00,51.0,dpp7-xgboost,NaN,NaN,NaN
43,0.000004,0.570909,0.647989,0.000046,0.000165,2.0,0.000050,156.0,0.866801,Autopilot--notebook--200301-1524-207-dc2d1714,Completed,0.985401,2020-03-01 16:57:47+00:00,2020-03-01 16:58:43+00:00,56.0,dpp7-xgboost,NaN,NaN,NaN
32,0.057709,0.918741,0.279328,0.000044,1.266111,6.0,0.000009,270.0,0.809746,Autopilot--notebook--200301-1524-218-acc68384,Completed,0.985401,2020-03-01 17:04:03+00:00,2020-03-01 17:05:06+00:00,63.0,dpp8-xgboost,NaN,NaN,NaN
169,0.000856,0.333097,0.128519,0.000001,0.348400,19.0,0.391675,248.0,0.557505,Autopilot--notebook--200301-1524-081-9bc25189,Completed,0.985401,2020-03-01 15:59:24+00:00,2020-03-01 16:00:21+00:00,57.0,dpp7-xgboost,NaN,NaN,NaN
17,0.000179,0.453178,0.163601,0.000007,0.000006,2.0,0.037952,128.0,0.972518,Autopilot--notebook--200301-1524-233-f1ffab6f,Completed,0.983942,2020-03-01 17:11:15+00:00,2020-03-01 17:12:04+00:00,49.0,dpp7-xgboost,NaN,NaN,NaN


The best training job can be selected as below:

<div class="alert alert-info"> 💡 <strong>Tips: </strong>
You could select alternative job by using the value from `TrainingJobName` column above and assign to `best_training_job` below
</div>

In [23]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)
best_training_job = attached_tuner.best_training_job()

print("Best Multi Algorithm HPO training job name is {}".format(best_training_job))

Best Multi Algorithm HPO training job name is Autopilot--notebook--200301-1524-125-8c267365


### Linking Best Training Job with Feature Pipelines

Finally, deploy the best training job to Amazon SageMaker along with its companion feature engineering models.
At the end of the section, you get an endpoint that's ready to serve online inference or start batch transform jobs!

Deploy a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/pipeline.html) that has multiple containers of the following:

1. Data Transformation Container: a container built from the model we selected and trained during the data transformer sections
2. Algorithm Container: a container built from the trained model we selected above from the best HPO training job.
3. Inverse Label Transformer Container: a container that converts numerical intermediate prediction value back to non-numerical label value.

Get both best data transformation model and algorithm model from best training job and create an pipeline model:

In [24]:
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel

# Get a data transformation model from chosen candidate
best_candidate = automl_interactive_runner.choose_candidate(df_tuning_job_analytics, best_training_job)
best_data_transformer_model = best_candidate.get_data_transformer_model(role=SAGEMAKER_ROLE, sagemaker_session=SAGEMAKER_SESSION)

# Our first data transformation container will always return recordio-protobuf format
best_data_transformer_model.env["SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT"] = 'application/x-recordio-protobuf'
# Add environment variable for sparse encoding
if best_candidate.data_transformer_step.sparse_encoding:
    best_data_transformer_model.env["AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF"] = '1'

# Get a algo model from chosen training job of the candidate
algo_estimator = Estimator.attach(best_training_job)
best_algo_model = algo_estimator.create_model(env={'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT':"text/csv"})

# Final pipeline model is composed of data transformation models and algo model and an
# inverse label transform model if we need to transform the intermediates back to non-numerical value
model_containers = [best_data_transformer_model, best_algo_model]
if best_candidate.transforms_label:
    model_containers.append(best_candidate.get_data_transformer_model(
        transform_mode="inverse-label-transform",
        role=SAGEMAKER_ROLE,
        sagemaker_session=SAGEMAKER_SESSION))

pipeline_model = PipelineModel(
    name="AutoML-{}".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name),
    role=SAGEMAKER_ROLE,
    models=model_containers,
    vpc_config=AUTOML_LOCAL_RUN_CONFIG.vpc_config)

2020-03-01 17:21:01,997 INFO root: Chosen Data Processing pipeline candidate name is dpp7-xgboost
2020-03-01 15:19:32 Starting - Preparing the instances for training
2020-03-01 15:19:32 Downloading - Downloading input data
2020-03-01 15:19:32 Training - Training image download completed. Training in progress.
2020-03-01 15:19:32 Uploading - Uploading generated training model
2020-03-01 15:19:32 Completed - Training job completed2020-03-01 15:19:10,116 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-03-01 15:19:10,118 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-01 15:19:10,128 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-03-01 15:19:10,345 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-03-01 15:19:10,345 sagemaker-containers INFO     Generating setup.cfg
2020-03-01 15:19:10,345 sagemaker-containers 

### Deploying Best Pipeline

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. You can customize the initial instance count and instance type used to deploy this model.
2. Endpoint name can be changed to avoid conflict with existing endpoints.

</div>

Finally, deploy the model to SageMaker to make it functional.

In [25]:
pipeline_model.deploy(initial_instance_count=1,
                      instance_type='ml.m5.2xlarge',
                      endpoint_name=pipeline_model.name,
                      wait=True)

2020-03-01 17:21:04,224 INFO sagemaker: Creating model with name: AutoML-Autopilot--notebook-run-01-15-10-53
2020-03-01 17:21:05,020 INFO sagemaker: Creating endpoint with name AutoML-Autopilot--notebook-run-01-15-10-53
-----------!

Congratulations! Now you could visit the sagemaker
[endpoint console page](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/endpoints) to find the deployed endpoint (it'll take a few minutes to be in service).

<div class="alert alert-warning">
    <strong>To rerun this notebook, delete or change the name of your endpoint!</strong> <br>
If you rerun this notebook, you'll run into an error on the last step because the endpoint already exists. You can either delete the endpoint from the endpoint console page or you can change the <code>endpoint_name</code> in the previous code block.
</div>